# Ajuste de hiperparametros

In [43]:

import numpy as np
import pandas as pd
import time
import warnings
from sklearn import metrics,preprocessing, tree
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [15]:
df= pd.read_csv('C:/Users/cbastian/Documents/MarchineLearning/python3/Bases del modelado/LoadPrediction/train_u6lujuX_CVtuZ9i.csv')
df.drop(['Loan_ID','Gender','Married','Dependents','Education','Self_Employed'],
axis='columns', inplace=True)
df.drop('Property_Area', axis='columns', inplace=True)
df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
0,5849,0.0,NaN,360.0,1.0,Y
1,4583,1508.0,128.0,360.0,1.0,N
2,3000,0.0,66.0,360.0,1.0,Y
3,2583,2358.0,120.0,360.0,1.0,Y
4,6000,0.0,141.0,360.0,1.0,Y


In [16]:
df.isnull().sum()

ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Loan_Status           0
dtype: int64

In [20]:
values = {'LoanAmount': 0.0, 'Loan_Amount_Term': 360, 'Credit_History': 1}
df.fillna(value=values, inplace=True)
df

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
0,5849,0.0,0.0,360.0,1.0,Y
1,4583,1508.0,128.0,360.0,1.0,N
2,3000,0.0,66.0,360.0,1.0,Y
3,2583,2358.0,120.0,360.0,1.0,Y
4,6000,0.0,141.0,360.0,1.0,Y
...,...,...,...,...,...,...
609,2900,0.0,71.0,360.0,1.0,Y
610,4106,0.0,40.0,180.0,1.0,Y
611,8072,240.0,253.0,360.0,1.0,Y
612,7583,0.0,187.0,360.0,1.0,Y


In [21]:
df.shape

(614, 6)

## usar arbol de desicion 

In [22]:
from sklearn.tree import DecisionTreeClassifier as dt
model= dt()

In [23]:
x= np.asanyarray(df.iloc[:,0:-1])
y= np.asanyarray(df.iloc[:,-1])

In [25]:
x.shape, y.shape

((614, 5), (614,))

In [28]:
X_train,X_test, y_train,y_test= train_test_split(x,y,test_size=0.20, random_state=0)

In [29]:
X_train.shape, X_test.shape

((491, 5), (123, 5))

## usar cross validation

In [32]:
#cv= lo partire en 5 
# score de clasificacion en este caso f1 macro
scores= cross_val_score(model,X_train,y_train, cv=5, scoring='f1_macro' )
print(scores)

[0.66243386 0.7030303  0.70185915 0.59894078 0.62582437]


In [34]:
## sacamos la media del score
## tenemos la medida robusca con cross validation 
scores.mean()

0.6584176933808502

## buscar los hiperparametros del modelo

 ### Busqueda por rejilla

In [35]:
# para el modelo de arbol de decision tengo algunos hiperparametros mas importantes
## maximo de profundidad= mayor prof mas complejo mi arbol de desicion
parameters= { 'max_depth':[1,2,3,4,5],
             ##minimo de muestras por hoja   
             'min_samples_leaf':[1,2,3,4,5],
             ## minimo de mestra por division
             'min_samples_split':[2,3,4,5],
             # y los criterios que pueden ser gini y entropy
             'criterion':['gini','entropy']
            }

In [37]:
## tratamos de mandar los mismos parametros que mandamos a nuestro cross validation

search_obj = GridSearchCV(model,  parameters, cv=5, scoring='f1_macro' )

## entrenamos nuestro objeto de busqueda entrena muchos modelos 
## cada modelo con las convinaciones posibles que mandamos
fit_obj= search_obj.fit( X_train,y_train )

##obtener los resultados del cros validation 
## cada numero es un hiperparametro que hicimos y lo esntrenamos todso a la vez
print(fit_obj.cv_results_['mean_test_score'])


[0.7268797  0.7268797  0.7268797  0.7268797  0.7268797  0.7268797
 0.7268797  0.7268797  0.7268797  0.7268797  0.7268797  0.7268797
 0.7268797  0.7268797  0.7268797  0.7268797  0.7268797  0.7268797
 0.7268797  0.7268797  0.71921059 0.71921059 0.71921059 0.71921059
 0.72272563 0.72272563 0.72272563 0.72272563 0.72272563 0.72272563
 0.72272563 0.72272563 0.72272563 0.72272563 0.72272563 0.72272563
 0.72272563 0.72272563 0.72272563 0.72272563 0.71652232 0.71652232
 0.71652232 0.71652232 0.69903543 0.69903543 0.69903543 0.69903543
 0.69903543 0.69903543 0.69903543 0.69903543 0.69738555 0.69738555
 0.69738555 0.69738555 0.70101339 0.69738555 0.70101339 0.70101339
 0.69889547 0.69889547 0.69889547 0.69889547 0.71355096 0.71355096
 0.71355096 0.71355096 0.69103056 0.70137041 0.69103056 0.70137041
 0.68209863 0.68572647 0.68209863 0.68572647 0.68572647 0.68572647
 0.68209863 0.68209863 0.66553143 0.6693941  0.6693941  0.6693941
 0.66856024 0.66856024 0.67923403 0.67923403 0.66604228 0.67578736

In [38]:
## el mejor estimador y lo guardamos
best_model= fit_obj.best_estimator_
best_model

DecisionTreeClassifier(max_depth=1)

In [49]:
#lo enrtrenare nuevamente con cross validation
scores= cross_val_score(best_model,X_train,y_train, cv=5,scoring='f1_macro' )
scores

array([0.76285755, 0.75168919, 0.76210196, 0.60606061, 0.75168919])

In [50]:
## sacar la media
scores.mean()

0.7268796972370323

In [51]:
## entrenarlo otra vez
best_model.fit(X_train, y_train)

## hacer las predicciones usando el nuevo modelo

print('Train: ', best_model.score(X_train,y_train))
print('Test: ', best_model.score(X_test,y_test))

Train:  0.8044806517311609
Test:  0.8292682926829268


### Busqueda Aleatoria

In [55]:

search_obj= RandomizedSearchCV(model, parameters, cv=5, scoring='f1_macro' )
fit_obj= search_obj.fit(X_train,y_train)
print(fit_obj.cv_results_['mean_test_score'])


[0.66044664 0.69903543 0.7268797  0.72272563 0.7268797  0.72272563
 0.67100228 0.70626423 0.7268797  0.72272563]


In [56]:
best_model= fit_obj.best_estimator_
best_model

DecisionTreeClassifier(criterion='entropy', max_depth=1, min_samples_split=4)

In [57]:
## lo entreno otra vezx

best_model.fit(X_train,y_train)

#hacemos las predicciones
print('Train: ', best_model.score(X_train,y_train))
print('Test: ', best_model.score(X_test,y_test))

Train:  0.8044806517311609
Test:  0.8292682926829268
